In [1]:
import h5py
import numpy as  np
from pathlib import Path

from source.preprocessHighFPS import yield_aggregate_bin_files, aggregate_bin_files, _to_analytic, _load_file, _chunk_to_fft, _get_radar_filenames

In [2]:
BASE_DIR = Path("E:/radar-project")  # change to your project dir
RADAR_DIR = BASE_DIR / "data/2023_03_21"  # day dir
GROUP_NAME = '2023_03_21' # nazwa grupy (np. dzień pomiaru), de facto to tylko kolejny wymiar
H5_FILENAME = 'alive_or_dead'

In [3]:
files = [
         # 'background',
         # 'wysoki_niski_3m_oskar_bartek_przodem',
         'wiatrak_4m',
         # 'wiatrak_2m_bartek_sciana',
         'wiatrak_2m',
         # 'wiatrak_3m',
         # 'wiatrak_1m',
         # 'randomowe_chodzonko_3os',
         'randomowe_chodzonko_1os_bartek',
         # 'odleglosc_4m_bartek_bokiem',
         # 'odleglosc_4m_bartek_przodem',
         # 'odleglosc_3m_bartek_bokiem',
         # 'odleglosc_3m_bartek_przodem',
         # 'odleglosc_2m_bartek_bokiem',
         # 'odleglosc_2m_bartek_przodem',
         # 'odleglosc_1m_bartek_bokiem',
         # 'odleglosc_1m_bartek_przodem',
         # 'chodzonko_2os_2m_oskar_3m_bartek',
         # 'chodzonko_2os_2m_bartek_3m_oskar',
         'chodzonko_1os_4-1-8m_bartek',
         # 'bartek_sciana',
         # 'biegunko_1os_2m_rafal',
         # 'chodzonko_1os_2m_bartek',
         ]

try:
    h5file = h5py.File('../data/'+ H5_FILENAME + '.h5','r+')
except:
    print('File not exist, creating file')
    h5file = h5py.File('../data/'+ H5_FILENAME + '.h5','w')

h5group = h5file.create_group(GROUP_NAME)

for idx, file in enumerate(files):
    data = yield_aggregate_bin_files('2023_03_21_' + file, 3, chunk_length=1, radar_dir=RADAR_DIR)

    h5dset =  h5group.create_dataset(name=file,
                                     shape=(1, 3, 4, 512),
                                     maxshape=(None,3,4,512),
                                     dtype= np.complex128,
                                     compression='gzip', shuffle=True)

    i = 0
    while True:
        try:
            chunk = next(data)
            chunk = _to_analytic(chunk)
        except ValueError:
            print('File converted')
            print(f'Chunks number: {i}')
            break

        if i != 0:
            h5dset.resize(h5dset.shape[0]+1, axis=0)

        h5dset[i,:,:,:] = chunk[0,:,:,:]
        i+=1

h5file.close()

File not exist, creating file


File converted
Chunks number: 13513


File converted
Chunks number: 16185


File converted
Chunks number: 19884


File converted
Chunks number: 15395


In [4]:
loaded_file = h5py.File('../data/'+ H5_FILENAME + '.h5','r')
frames = loaded_file[GROUP_NAME]['wiatrak_4m']

In [5]:
frames[0]

array([[[  23.+168.j, -105. +53.j, -121. -17.j, ...,  -24. +19.j,
          -30. +16.j,  -36.  -6.j],
        [-175.-101.j,  -48.+103.j,  -92. +84.j, ...,   29. -21.j,
           38. +25.j,   15. +51.j],
        [ -96. +91.j,  -94.+129.j, -127.+108.j, ...,    5. -32.j,
           58. -14.j,   44. +30.j],
        [-159.+292.j,  171.+163.j,  140.+169.j, ...,  -44. +14.j,
          -20.  -3.j,    4. -12.j]],

       [[ 127. +45.j,  -47.  -8.j,  -58. -26.j, ...,  -25. -53.j,
           21. -65.j,   60. -25.j],
        [-161.+151.j,  129.+145.j,  111.+167.j, ...,  -59.  -5.j,
          -32. -17.j,   13. -17.j],
        [  80. +94.j,  -49. +10.j,  -45.  -6.j, ...,  -40. -69.j,
           31. -80.j,   57. -56.j],
        [ -79.+147.j,  246. +36.j,  257. +87.j, ...,  -91. -39.j,
          -45. -72.j,   -2. -86.j]],

       [[ 109.+171.j,  -67. -84.j,  -27.-111.j, ...,  -22. -60.j,
           32. -55.j,   65. -30.j],
        [-203. -85.j,   -6. +87.j,  -31. +66.j, ...,  -17. -49.j,
            

In [6]:
loaded_file.close()